In [17]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.ops import cascaded_union


import matplotlib.pyplot as plt
import geopandas as gpd
import pysal as ps

#plot map of grunnkrets
def plot_gk():
    df = pd.read_csv('data/grunnkrets_norway_stripped.csv')
    df = gpd.GeoDataFrame(df)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.plot(figsize=(20, 20))
    plt.show()

    #plot map of busstops
    df = pd.read_csv('data/busstops_norway.csv')
    df = gpd.GeoDataFrame(df)
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    df = gpd.GeoDataFrame(df, geometry='geometry')
    df.plot(figsize=(20, 20))
    plt.show()



train_data = pd.read_csv('data/stores_train.csv', nrows= 1000)
place = pd.read_csv('data/plaace_hierarchy.csv', nrows= 1000)
train_data = pd.merge(train_data, place[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data.head()


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,lv1,lv2,lv3
0,983540538-974187930-44774,2016.0,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303.0,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,1,1.1,1.1.1
1,987074191-973117734-44755,2016.0,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306.0,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,1,1.1,1.1.1
2,984890265-981157303-64491,2016.0,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102.0,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,1,1.1,1.1.1
3,914057442-992924179-126912,2016.0,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102.0,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,1,1.1,1.1.1
4,913018583-913063538-668469,2016.0,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414.0,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,1,1.1,1.1.1


In [18]:
#number of stores within x radius
import math
import geopy.distance


df = train_data
df=df.loc[:,['store_id','lat','lon', 'lv2']]

df['num_stores_within_distance'] = 0
def stores_within_distance(distance):
    for index in range(len(df)):
        lvl1 = df._get_value(index,'lv2')
        lat1 = df._get_value(index,'lat')
        lon1 = df._get_value(index,'lon')
        count = 0
        for inx in range(len(df)):
            lvl2 = df._get_value(inx,'lv2')
            lat2 = df._get_value(inx,'lat')
            lon2 = df._get_value(inx,'lon')
            try:
                dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
                print(lat1, lat2, lon1, lon2)
            except:
                print(lat1, lat2, lon1, lon2)

            if dist < distance and lvl1 == lvl2:
                count += 1
        df._set_value(index, 'num_stores_within_distance', count- 1)


#stores_within_distance(0.1)



#df.head(20)


In [19]:
#distance from shop to closest buss station
from dis import dis
import math
import geopandas
import numpy
from shapely import wkt
from shapely import wkb

#busstops
gdf = gpd.GeoDataFrame(pd.read_csv('data/busstops_norway.csv', nrows=100))
gdf=gdf.loc[:,['importance_level', 'geometry']]
gdf['geometry'] = gpd.GeoSeries.from_wkt(gdf['geometry'])

# stores train
df['distance_to_closest_buss_station'] = 0
df['importance_level'] = 0
df['closest_local_knutepunkt'] = 0
df = train_data

#add to dataframe
for index in range(len(gdf)):
    closest = 100000 #large number
    importance = ""
    closest_high_importance = 0
    lat = df._get_value(index,'lat')
    lon = df._get_value(index,'lon')
    for numbuss in range(len(gdf)):
        val = gdf._get_value(index,'geometry')
        dist = geopy.distance.geodesic((val.x, val.y), (lat, lon)).km
        if dist < closest:
            closest = dist
            importance = gdf._get_value(index,'importance_level')
        if gdf._get_value(index,'importance_level') == 'Lokalt knutepunkt':
            closest_high_importance = dist
    df._set_value(index, 'distance_to_closest_buss_station', closest)
    df._set_value(index, 'importance_level', importance)
    df._set_value(index, 'closest_local_knutepunkt', closest_high_importance)

df.head()






,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,lv1,lv2,lv3,distance_to_closest_buss_station,importance_level,closest_local_knutepunkt
0,983540538-974187930-44774,2016.0,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303.0,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,1,1.1,1.1.1,7279.348126,Mangler viktighetsnivå,0.0
1,987074191-973117734-44755,2016.0,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306.0,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,1,1.1,1.1.1,7285.624462,Mangler viktighetsnivå,0.0
2,984890265-981157303-64491,2016.0,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102.0,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,1,1.1,1.1.1,6825.193589,Mangler viktighetsnivå,0.0
3,914057442-992924179-126912,2016.0,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102.0,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,1,1.1,1.1.1,7113.556468,Mangler viktighetsnivå,0.0
4,913018583-913063538-668469,2016.0,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414.0,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,1,1.1,1.1.1,7009.178511,Mangler viktighetsnivå,0.0


In [33]:
# make postal code per store 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")


# Display
train_data = pd.read_csv('data/stores_train.csv', nrows= 10)
df = train_data
df['postcode'] = 0
for index in range(len(df)):
    lat = df._get_value(index,'lat')
    lon = df._get_value(index,'lon')
    location = geolocator.reverse(str(lat)+","+str(lon))
    address = location.raw['address']
    zipcode = address.get('postcode')
    df._set_value(index, 'postcode', str(zipcode))
df['postcode']= df.postcode.astype('category')
df.head(10)

13, Bragernes torg, Bragernes, Drammen, Viken, 3017, Norge
4, Klingenberggata, Tullin, Sentrum, Oslo, 0161, Norge
Kuben Senter, Stangs gate, Hønefoss, Ringerike, Viken, 3510, Norge
Glasshuset, Dronningens gate, Bodø, Bodø sentrum, Bodø, Nordland, 8005, Norge
Tillertorget, Ivar Lykkes veg, Tillerbyen, Tiller, Heimdal, Trondheim, Trøndelag, 7072, Norge
Burger King, Torget, Kannik, Storhaug, Eiganes og Våland, Stavanger, Rogaland, 4006, Norge
Amfi Narvik, Sleggesvingen, Luossavaara Kiirunavaara Aktie Bolag (LKAB), Trekanten, Narvik - Áhkánjárga, Narvik, Nordland, 8516, Norge
1, Olav Vs gate, Knardal, Halden, Viken, 1767, Norge
Arken, E 16;E 39, Nyborg, Åsane, Bergen, Vestland, 5116, Norge
2, Heddalsvegen, Tinnesmoen, Tinnes søndre, Notodden, Vestfold og Telemark, 3674, Norge


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,postcode
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,3017
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,161
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,3510
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,8005
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,7072
5,998035295-979540094-54657,2016,BURGER KING STAVANGER,1.1.1.0,Hamburger restaurants,11030509,TORGET 7,58.970249,5.731728,BURGER KING,NaN,12.020,4006
6,916757433-916772262-822439,2016,BURGER KING NARVIK,1.1.1.0,Hamburger restaurants,18050706,BOLAGSGATA 1,68.438025,17.422916,BURGER KING,Amfi Narvik,7.622,8516
7,913158733-913203305-675470,2016,GRENSEN BURGERBAR,1.1.1.0,Hamburger restaurants,1010301,OLAV VS GATE 1,59.119175,11.389572,NaN,NaN,3.640,1767
8,982174139-982239737-14482,2016,MCDONALD'S ÅSANE SENTER,1.1.1.0,Hamburger restaurants,12010915,ÅSANE SENTER 37,60.465118,5.324101,MCDONALDS,Åsane Storsenter,30.514,5116
9,916217153-911834553-435311,2016,BERGBYS NOTODDEN,1.1.1.0,Hamburger restaurants,8070301,NESØYA,59.559079,9.249380,BERGBYS,NaN,6.339,3674
